## Preprocessing ...

## SVM

In [2]:
import numpy as np
import os
from PIL import Image
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler

# 1. Lade die Daten aus den Ordnern
# Ordnerpfad definieren
data_path = r"C:\Users\Flavio\Bachelorarbeit\LSB\data_LSB"
clean_path = os.path.join(data_path, "clean")
stego_path = os.path.join(data_path, "stego")

# Funktionen zum Laden der Bilder
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        try:
            with Image.open(img_path) as img:
                images.append(np.array(img))
        except Exception as e:
            print(f"Fehler beim Laden von {img_path}: {e}")
    return images

# Bilder und Labels laden
clean_images = load_images_from_folder(clean_path)
stego_images = load_images_from_folder(stego_path)

# Konvertiere zu numpy Arrays und erstelle Labels
X = np.array(clean_images + stego_images)
y = np.array([0] * len(clean_images) + [1] * len(stego_images))  # 0 = clean, 1 = stego

# 2. Reshape der Daten
# SVM benötigt die Daten in flacher Form, also (n_samples, n_features).
X_flat = X.reshape(X.shape[0], -1)

# 3. Train-Test-Split
X_train, X_test, y_train, y_test = train_test_split(X_flat, y, test_size=0.2, random_state=42)

# 4. Daten-Normalisierung
# StandardScaler skaliert die Daten auf Mittelwert=0 und Varianz=1, was für SVM wichtig ist.
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 5. SVM-Definition und Hyperparameter-Tuning
# GridSearchCV wird verwendet, um die besten Hyperparameter zu finden.
param_grid = {
    'C': [0.1, 1, 10, 100],  # Regularisierungsparameter
    'gamma': ['scale', 'auto', 0.01, 0.001, 0.0001],  # Kernel-Koeffizient
    'kernel': ['rbf']  # Radial Basis Function Kernel
}

svc = SVC()
grid_search = GridSearchCV(svc, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=-1)
grid_search.fit(X_train_scaled, y_train)

# 6. Beste Parameter und Modellbewertung
best_svm = grid_search.best_estimator_
print("Beste Parameter:", grid_search.best_params_)

# 7. Evaluierung des Modells
# Testvorhersagen
y_pred = best_svm.predict(X_test_scaled)

# Ergebnisse ausgeben
print("\nKlassifikationsbericht:")
print(classification_report(y_test, y_pred))

print("\nKonfusionsmatrix:")
print(confusion_matrix(y_test, y_pred))


Fitting 5 folds for each of 20 candidates, totalling 100 fits
Beste Parameter: {'C': 1, 'gamma': 0.0001, 'kernel': 'rbf'}

Klassifikationsbericht:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6969
           1       1.00      1.00      1.00      7031

    accuracy                           1.00     14000
   macro avg       1.00      1.00      1.00     14000
weighted avg       1.00      1.00      1.00     14000


Konfusionsmatrix:
[[6967    2]
 [   6 7025]]


In [3]:
from joblib import dump

# Speichern des besten SVM-Modells
dump(best_svm, "svm_model.joblib")
print("Das Modell wurde erfolgreich gespeichert.")


Das Modell wurde erfolgreich gespeichert.


In [ ]:
from joblib import load

# Laden des gespeicherten Modells
loaded_svm = load("svm_model.joblib")
print("Das Modell wurde erfolgreich geladen.")

# Beispiel: Vorhersagen mit geladenem Modell
y_pred = loaded_svm.predict(X_test_scaled)
